# Mainly for model training

Depending on the size of your training set, you will need an [inference notebook](https://www.kaggle.com/code/regisvargas/inference-jane-street-a-beginner-s-notebook).

In [ ]:
import pandas as pd
# Initialize a list to hold samples from each file
samples = []
# Load a sample from each file
for i in range(10):
    file_path = f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
    chunk = pd.read_parquet(file_path)
    
    # Take a sample of the data (adjust sample size as needed)
    sample_chunk = chunk.sample(n=500000, random_state=42)  # For example, 100 rows
    samples.append(sample_chunk)
# Concatenate all samples into one DataFrame if needed
sample_df = pd.concat(samples, ignore_index=True)


In [ ]:
sample_df.head()

# Prepare data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
# Separate features and responders
features = sample_df.filter(regex='^feature_')
responders = sample_df.filter(regex='^responder_')
weights = sample_df['weight']
# Convert to numpy arrays for TensorFlow
X = features.values  # Features for input
#y = responders.values  # Responders for output
# Assuming you have a DataFrame `y_train` with all responders
y = responders[['responder_6']].values  # Keep only responder_6
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

In [ ]:
Is_keras = False

# XGBoost

See [Feature engineering, xgboost](https://www.kaggle.com/code/dlarionov/feature-engineering-xgboost#Part-2,-xgboost) for details.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val, weights_train, weights_val = train_test_split(
    X, y, weights, test_size=0.2, random_state=42
)

In [ ]:
# Define a learning rate schedule
def learning_rate_scheduler_xgb(epoch):
    initial_rate = 0.3
    decay_rate = 0.999
    return initial_rate * (decay_rate ** (np.log(epoch)))

In [ ]:
from xgboost import XGBRegressor
# Create an XGBoost model
model_xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=learning_rate_scheduler_xgb,
    max_depth=8,
    random_state=42
)
# Fit the model with sample weights and validation dataset
model_xgb.fit(
    X_train,
    y_train,
    sample_weight=weights_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    sample_weight_eval_set=[weights_train, weights_val],
    eval_metric='rmse',
    early_stopping_rounds=10,
    verbose=True
)


# Build the Autoencoder Model

Gradient Centralization for Better Training Performance

See https://keras.io/examples/vision/gradient_centralization/ for details. 

In [ ]:
from keras.optimizers import RMSprop
class GCRMSprop(RMSprop):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.
        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= ops.mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)
        return grads
optimizer = GCRMSprop(learning_rate=1e-4)

In [ ]:
from tensorflow.keras.regularizers import l2
# Define the number of input and output nodes
input_dim = X.shape[1]  # Number of features (79)
output_dim = y.shape[1]  # Number of responders (9)
# Define the model
model = models.Sequential([
    layers.Input(shape=(input_dim,)), # Input layer
    layers.LayerNormalization(),
   # layers.BatchNormalization(),
  #  layers.Dense(128, activation='relu'),
  #  layers.Dropout(0.2),
    layers.Dense(64, activation='linear'),  # Encoder
    layers.Dense(32, activation='linear'),  # Bottleneck layer (compression)
    layers.Dense(64, activation='linear'),  # Decoder
#    layers.Dense(128, activation='relu'), 
 #   layers.Dropout(0.2),
    layers.Dense(output_dim, activation='linear', kernel_regularizer=l2(0.001))  # Output layer for responders
])
model.compile(optimizer="adam", loss='mse')

# Train Autoencoder Model

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=10,            # Number of epochs to wait for improvement
    min_delta=0.00001,       # Minimum change to qualify as an improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

if Is_keras:
    history = model.fit(
        X, y,
        epochs=50,
        verbose=0,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping,reduce_lr]
    )

In [ ]:
if Is_keras:
    model.save("/kaggle/working/model.keras")

# Submission

See [Jane Street RMF Demo Submission](https://www.kaggle.com/code/ryanholbrook/jane-street-rmf-demo-submission) for details.

Depending on the size of your training set, you will need an [inference notebook](https://www.kaggle.com/code/regisvargas/inference-jane-street-a-beginner-s-notebook).

In [ ]:
import os
import polars as pl
import kaggle_evaluation.jane_street_inference_server

In [ ]:
import polars as pl
import numpy as np
# Assuming `model` is your trained model
# Assuming features required by the model are named 'feature_00', 'feature_01', etc.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags
    # Extract the features for the model input
    feature_columns = [col for col in test.columns if col.startswith("feature_")]
    features = test.select(feature_columns).to_numpy()  # Convert to numpy array for model input
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    # Generate predictions using the model
    #model_predictions = model.predict(features)
    if Is_keras:
        responder_6_predictions = model.predict(features)[:,0]
    else:
        responder_6_predictions = model_xgb.predict(features)
   # print(responder_6_predictions)    
    #responder_6_predictions = model_predictions[:, 6]  # Assuming responder_6 is at index 6
    # Create a new Polars DataFrame with row_id and responder_6 predictions
    predictions = test.select("row_id").with_columns(
        pl.Series("responder_6", responder_6_predictions)
    )
    print(predictions)
    # Ensure the output format and length requirements
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    
    assert len(predictions) == len(test)
    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )